# <p style="text-align: center;"> Variational Quantum Linear Solver on Rigetti QCS </p>

<p style="text-align: center;"> Ryan LaRose, Department of Computational Mathematics, Science, and Engineering & Department of Physics and Astronomy, Michigan State University. </p>

<img src="main.png" alt="Overview of VQLS">

### <p style="text-align: center;"> Abstract </p>

<p style="text-align: justify;"> The variational quantum linear solver (VQLS) [1] is an algorithm for solving linear systems on near-term quantum computers. While VQLS does not offer performance guaruntees like other quantum linear systems algorithms [2-3], the ability to run on near-term computers makes it an interesting problem-specific benchmark. In this notebook, we provide a brief tutorial of VQLS and show results of running VQLS on Rigetti QCS for example linear systems. </p>

## <p style="text-align: center;"> Notebook Setup </p>

<p style="text-align: justify;"> We use pyQuil to write the VQLS algorithm and send job submissions to Rigetti QCS. In the notebook, we use the Rigetti QVM for tutorial purposes. When running on the device, the code written in the notebook is converted to a script and executed on the QPU. </p>

In [2]:
"""Imports."""
from itertools import product
from math import pi
import time

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.optimize import (dual_annealing, fmin, fmin_powell, fmin_cobyla, minimize)

import pyquil
from pyquil import Program, get_qc
import pyquil.gates as gates

from aqgd import AQGD
import vqls

In [3]:
"""Constants and other setup."""
np.set_printoptions(precision=3)
%matplotlib inline

# <p style="text-align: center;"> The VQLS Algorithm </p>

A linear system $A \mathbf{x} = \mathbf{b}$ is defined by a matrix $A \in \mathbb{C}^{N \times N}$ and vector $\mathbf{b} \in \mathbb{C}^{N}$. Given $A$ and $\mathbf{b}$ as input, the goal is to output $\mathbf{x} \in \mathbb{C}^{N}$, or an approximate solution $\mathbf{\bar{x}}$ where $|| \mathbf{\bar{x}} - \mathbf{x} || \le \delta$. 

### <p style="text-align: center;"> Problem Setup </p>

<p style="text-align: justify;"> How can we solve linear systems of equations with a (near-term) quantum computer? Without loss of generality, we can express the matrix $A$ is a linear combination of Paulis</p>

\begin{equation}
    A = \sum_{l = 1}^{L} a_l A_l .
\end{equation}

<p style="text-align: justify;"> The vector $|\mathbf{b}\rangle$ is assumed to have some unitary $B$ which prepares it from the ground state, i.e. $B|0\rangle = |\mathbf{b}\rangle$. We can also take $B = \sum_{m = 1}^{M} b_m B_m$ as a linear combination of Paulis. </p>

We want to prepare a state

\begin{equation}
    |\mathbf{\theta}\rangle := V(\mathbf{\theta}) |0\rangle 
\end{equation}

<p style="text-align: justify;"> which has high fidelity $| \langle \theta | \mathbf{x} \rangle |^2$ with the solution $|x\rangle$ of the linear system $A |\mathbf{x}\rangle = |\mathbf{b}\rangle $. Equivalenlty, we want a state $|\theta\rangle$ such that $A |\theta\rangle$ is close to $|\mathbf{b}\rangle$. By maximizing the fidelity $|\langle \mathbf{b} | A |\theta\rangle|^2$ where

\begin{equation}
    \langle \mathbf{b} | A |\theta\rangle = \langle 0 | B^\dagger A V(\theta) |0\rangle = \sum_{l = 1}^{L} c_l \langle 0 | B^\dagger A_l V(\theta) |0\rangle .
\end{equation}

one can obtain an approximate solution $|\theta_\text{opt}\rangle = V(\theta_\text{opt}) |0\rangle$ to the linear system. 

One way to compute the fidelity is via the Hadamard Test or extensions of the Hadamard Test [1]. Another way is described below.

### <p style="text-align: center;"> The Effective Hamiltonian Approach </p>

An interesting connection to other linear systems work [4] and variational algorithms literature [5] is to note that the above prescription is equivalent to minimizing the energy of an effective Hamiltonian

\begin{equation}
    H_{A, \mathbf{b}} \equiv H := A^\dagger ( I - |\mathbf{b}\rangle \langle \mathbf{b} | ) A
\end{equation}

<p style="text-align: justify;">
This can be seen by noting that $H$ is positive semidefinite and $H |\mathbf{x}\rangle = 0$. Thus, by minimizing $\langle \theta | H |\theta \rangle$, we can get an approximate solution to the linear system. A key advantage of this approach is the ability to compute expectation values in a hardware efficient manner. (That is, not using controlled gates required by the Hadamard test but instead using classical post-processing [5-7].) We use this <b>effective Hamiltonian approach</b> in our simulations below.
</p>

Noting that $H = A^\dagger A - A^\dagger P_{\mathbf{b}} A$ where $P_{\mathbf{b}} := |\mathbf{b}\rangle \langle \mathbf{b} |$ is the projector onto the $|\mathbf{b}\rangle$ state, we have two terms to evaluate. For the first, one can show that

\begin{equation}
    A^\dagger A = \sum_{l = 1}^{L} |a_l|^2 I + 2 \sum_{1 = l < k}^{L - 1} \text{Re} \, [a_l^* a_k ] A_l A_k .
\end{equation}

The expectation of the first summation can be computed classically, while the second requires $L (L - 1) / 2 = O(L^2)$ circuit evaluations. 

For the second term in the Hamiltonian $H$, one can see that

\begin{equation}
    A^\dagger P_{\mathbf{b}} A = \sum_{k = 1}^{L} \sum_{l = 1}^{L} a_k^* a_l A_k P_\mathbf{b} A_l .
\end{equation}

Assuming $B = \sum_{m = 1}^{M} b_m B_m$, the projector $P_\mathbf{b} = B|0\rangle \langle 0 | B^\dagger$ can be expanded as a linear combination of unitaries as well. Note that $|0\rangle\langle0| = (I + Z) / 2$. In general, there are $N M^2 L^2$ weighted Pauli operators in this term ($N = 2^n$ where $n$ is the number of qubits).  

<p style="text-align: justify;">
As mentioned, the <font color="green"><b>advantage</b></font> of this approach is the short-depth circuits for cost evaluation. The <font color="red"><b>disadvantage</b></font> of this approach is that it can lead to many terms in the Pauli expansion of $H_{A, \mathbf{b}}$. An interesting observation is the "physicality" of this effective Hamiltonian -- namely, physical Hamiltonians usually have (or are assumed to have) poly($n$) terms, while this Hamiltonian has $O(N)$ terms in the worst case. We remark that simplifications which reduce the number of terms in the effective Hamiltonian $H_{A, \textbf{b}}$ are usually possible. Additionally, computing $\langle H \rangle$ by exploiting simulataneous measurements of observables can mitigate the large number of terms. Finally, we remark that coefficients in the projector $|0\rangle \langle 0|$ on $n$ qubits are equal to $1 / 2^n$, and it can be a reasonable approximation to skip terms with small coefficients. </p>

# <p style="text-align: center;"> Linear System 1 </p>

<p style="text-align: justify;"> The first linear system we consider is an example from the VQLS paper. Our motivation for this is to compare results from the paper (which used older Rigetti computers) to results we get on the new computers. The linear system is a three- to five-qubit example given by </p>

\begin{equation}
    A_1 = I + 0.2 X_1 + 0.2 X_1 Z_2 .
\end{equation}

and $\mathbf{b}_1 = H_1 H_2 H_4 H_4 |0\rangle^{\otimes n}$. Here, $I, X, Y$ and $Z$ are the usual Pauli operators

\begin{equation}
    I := \left[ \begin{matrix}
    1 & 0 \\
    0 & 1 \\
    \end{matrix} \right], \qquad 
    X := \left[ \begin{matrix}
    0 & 1 \\
    1 & 0 \\
    \end{matrix} \right], \qquad 
    Y := \left[ \begin{matrix}
    0 & -i \\
    i & 0 \\
    \end{matrix} \right], \qquad 
    Z := \left[ \begin{matrix}
    1 & 0 \\
    0 & -1 \\
    \end{matrix} \right]
\end{equation}

and subscripts indicate which qubits the Paulis act on. For example, $X_1$ means $X$ on the first qubit, i.e. $XII$ on three qubits or $X I I I I$ on five qubits.


For concreteness, we can reduce to three qubits and view the $8 \times 8$ matrix of this linear system. (Of course this works for any number of qubits, but we quickly run out of room on the screen!)

In [4]:
"""View the matrix of a linear system."""
# Define the matrix of the linear system as weighted Paulis
Acoeffs = [1., 0.2, 0.2]
Aterms = ["III", "XII", "XZI"]

# Display the matrix
print("The matrix of the linear system is:")
print(np.real(vqls.matrix(Acoeffs, Aterms)))

The matrix of the linear system is:
[[1.  0.  0.  0.  0.4 0.  0.  0. ]
 [0.  1.  0.  0.  0.  0.4 0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  1.  0.  0.  0.  0. ]
 [0.4 0.  0.  0.  1.  0.  0.  0. ]
 [0.  0.4 0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  1. ]]


<font color="blue"><b>Note:</b></font> The matrix is quite sparse. This is expected for linear systems with only a few weighted Paulis, as Pauli strings are sparse matrices. 

Using the identity $H = (I + Z) / \sqrt{2}$, we can expand the $B$ unitary as 

\begin{align}
    B &= \frac{1}{2^2} (I + Z) \otimes (I + Z) \otimes I \otimes (I + Z) \otimes (I + Z) \\
      &= \frac{1}{2^2} \left[ IIIII + IIIIZ + IIIZI + IIIZZ + IZIII + IZIIZ + IZIZI + IZIZZ + \cdots \right]
\end{align}

## <p style="text-align: center;"> Three Qubit Tutorial </p>

Below we consider the three-qubit verstion of the linear system above. First, we define the linear system in terms of $A$ and $B$, then compute the effective Hamiltonian of the linear system. 

In [5]:
"""Terms in effective Hamiltonian for first linear system."""
# Number of qubits
n = 3

# Define the B matrix of the linear system
Bcoeffs = [1 / 2**(n / 2)] * 2**n
paulis = [["X", "Z"]] * n
prods = list(product(*paulis))
Bterms = ["".join(p) for p in prods]

# Define the A matrix of the linear system
Acoeffs = [1, 0.2, 0.2]
Aterms = ["III", "XII", "XZI"]

# Get the effective Hamiltonian
ham = vqls.effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)

# Display the effective Hamiltonian
nterms = len(ham)
print("Number of terms:", nterms)
print("H = ")
for ii in range(nterms):
    ham[ii][0] = np.real(ham[ii][0])
    print(f"{round(ham[ii][0], 3)} \t* \t{ham[ii][1]}", (lambda ii: "+" if ii < nterms - 1 else "")(ii))

Number of terms: 12
H = 
0.895 	* 	III +
0.215 	* 	XII +
0.34 	* 	XZI +
0.02 	* 	IZI +
-0.185 	* 	IIX +
-0.175 	* 	IXI +
-0.06 	* 	IZX +
-0.175 	* 	IXX +
-0.185 	* 	XIX +
-0.175 	* 	XXI +
-0.06 	* 	XZX +
-0.175 	* 	XXX 


To evaluate these terms with a quantum computer, we first need a quantum computer! Below we select an appropriate `Aspen` lattice.

In [6]:
"""Get a quantum computer to run on."""
qcomputer = f"Aspen-7-{n}Q-B"  # Three qubit lattice for testing optimizer
lattice = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f37bf3f83c8>: Failed to establish a new connection: [Errno 111] Connection refused',)': /
WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f37bf3f8828>: Failed to establish a new connection: [Errno 111] Connection refused',)': /
WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f37bf3f8a20>: Failed to establish a new connection: [Errno 111] Connection refused',)': /


QVMNotRunning: No QVM server running at http://127.0.0.1:5000

We can visualize the connectivity of this lattice by drawing the qubit topology. 

In [ ]:
"""Visualize the qubit connectivity."""
graph = lattice.qubit_topology()
nx.draw(graph)

### <p style="text-align: center;"> Variational Ansatz </p>

The next and final ingredient we need to compute $\langle \theta | H | \theta \rangle$ is an ansatz state $|\theta\rangle$. We take this to be a product of $Y$-rotations on each qubit in the computer, obtained via `vqls.yansatz`.

In [ ]:
"""Get an ansatz and visualize it."""
circ, creg = vqls.yansatz(lattice)
print(circ)

As we see, the main circuit is simply $Y$-rotations on each qubit where the parameters `theta[0], theta[1], theta[2]` are independent.

The following function defines the cost we want to minimize by varying the parameters in the ansatz. The cost is exactly the energy of the effective Hamiltonian $H_{A, \mathbf{b}}$ computed via `vqls.energy`. 

In [ ]:
# TODO: Figure out why expectation value with type(angles) == np.array explodes (report bug)
# angles = np.array([1, 2, 3])
# angles = list(angles)
# vqls.expectation(angles, ham[1][0], ham[1][1], circ, creg, lattice, verbose=True)

In [ ]:
"""Cost function for linear system 1."""
def costLS1(angles, verbose: bool = False):
    val = vqls.energy(angles, ham, circ, creg, lattice, shots=10000)
    if verbose:
        print("Current angles:", angles)
        print("Current energy:", val)
    return val

In [11]:
"""Optimize by gradient descent."""
optimizer = AQGD(maxiter=25, tol=1e-3, eta=2.0, momentum=0.25, disp=True)
start = time.time()
res = optimizer.optimize([0] * n, costLS1)
print("Runtime:", (time.time() - start) // 60, "minutes.")

Iteration: 0	| Energy: 0.9086789999999995	| Params: [0, 0, 0]


KeyboardInterrupt: 

In [ ]:
"""Optimization: Testing fmin functions from scipy.optimize."""
start = time.time()
res = fmin(costLS1, x0=[pi / 2] * n, ftol=0.001, maxfun=50, args=(True,))
print("Runtime:", (time.time() - start) // 60, "minutes.")

In [ ]:
"""Do the optimization."""
# start = time.time()
# res = minimize(costLS1, x0=[pi / 2] * n, method="COBYLA", args=(True,))
# print("Runtime:", (time.time() - start) // 60, "minutes.")

In [ ]:
"""Optimize by annealing."""
# start = time.time()
# res = dual_annealing(costLS1, bounds=[(0, 2 * pi)] * n, args=(True,))
# print("Runtime:", (time.time() - start) // 60, "minutes.")

In [ ]:
"""Compute energy at optimal angles."""
vqls.energy([1.61, 1.88, 1.61], ham, circ, creg, lattice)

### <p style="text-align: center;"> Compare the Quantum and Classical Solutions </p>

In [ ]:
print("Minimum cost found:", res.fun)
print("Best found angles:", res.x)

In [ ]:
qxvec = vqls.qsolution(circ, res)
print(qxvec)

In [ ]:
"""Compare to classical solution."""
# Get the classical solution
Amat = vqls.matrix(Acoeffs, Aterms)
bvec = vqls.vector(Bcoeffs, Bterms)
xvec = np.linalg.solve(Amat, bvec)
xvec /= np.linalg.norm(xvec)

# ===========================
# Compare classical x vectors
# ===========================

print("Comparing x vectors:")
print("====================")

# Compute the fidelity
fidelity = abs(np.dot(qxvec.conj(), xvec))**2
print(r"| < xquantum | xclassical > |^2 =", round(fidelity, 2))

# Compute the L2 distance
l2dist = np.linalg.norm(qxvec - xvec, ord=2)**2
print(r"|| xquantum - xclassical ||^2 =", round(l2dist, 2))

# ==========================
# Compare solution b vectors
# ==========================

print("\nComparing b vectors:")
print("====================")

# Compute |qbvec> = A |theta optimal>
qbvec = Amat @ qxvec
qbvec /= np.linalg.norm(qbvec)
bfidelity = abs(np.dot(bvec.conj(), qbvec))**2
print(r"| < bquantum | bclassical > |^2 =", round(bfidelity, 3))

bl2dist = np.linalg.norm(qbvec - bvec, ord=2)**2
print(r"|| bquantum - bclassical ||^2 =", round(bl2dist, 3))

**SAVE THIS CELL!**

In [ ]:
"""SAVE THIS CELL!

Optimal angles for Ry product ansatz. Energy of effective Hamiltonian is near 0.001 here.
"""
LS1opt_angles = [1.60380032, 1.88421818, 1.61131353, 1.51679728, 1.44425814]

## Simultaneous measurements

In [12]:
"""Compare runtime for individual expectations vs simultaneous measurements."""
# Define number of qubits
n = 10

# Get a group to measure
group = [(1, "IIIIZIIIIZ"), (1, "IIIIIIZZII"), (1, "ZZZZZZZZZI"),
         (1, "IIIZZZIIIZ"), (1, "ZZIZZIZZZZ"), (1, "ZZZIZZZIZZ"),
         (1, "IIZZZIZZIZ"), (1, "ZZIZIIZIIZ"), (1, "ZZZZZZZZIZ"),
         (1, "ZZZIIIIIIZ"), (1, "IIIZIIIIZI"), (1, "ZIZZIIZZIZ")]

# Get a quantum computer to run on
qcomputer = f"Aspen-7-{n}Q-B"
lattice = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

# Get an ansatz and set the angles
circ, creg = vqls.yansatz(lattice)
angles = [0] * n

print(circ)

DECLARE ro BIT[10]
DECLARE theta REAL[10]
RY(theta[0]) 20
RY(theta[1]) 21
RY(theta[2]) 27
RY(theta[3]) 30
RY(theta[4]) 31
RY(theta[5]) 32
RY(theta[6]) 33
RY(theta[7]) 35
RY(theta[8]) 36
RY(theta[9]) 37



In [13]:
"""Measure each expectation value individually."""
start = time.time()
tot = 0.
for coeff, pauli in group:
    tot += vqls.expectation(angles, coeff, pauli, circ, creg, lattice, shots=10_000)
print(tot)
print("Runtime:", round(time.time() - start, 3), "seconds.")

12.0
Runtime: 40.864 seconds.


In [14]:
"""Measure the group."""
start = time.time()
gtot = vqls.measure_group(angles, group, circ, creg, lattice, shots=10_000)
print(gtot)
print("Runtime:", round(time.time() - start, 3), "seconds.")

12.0
Runtime: 4.156 seconds.


## <p style="text-align: center;"> Five Qubit Example </p>

In [ ]:
"""Terms in effective Hamiltonian for first linear system."""
# Number of qubits
n = 5

# Define the B matrix of the linear system
Bcoeffs = [1 / 2**(n / 2)] * 2**n
paulis = [["X", "Z"]] * n
prods = list(product(*paulis))
Bterms = ["".join(p) for p in prods]

# Define the A matrix of the linear system
Acoeffs = [1, 0.2, 0.2]
Aterms = ["IIIII", "XIIII", "XZIII"]

# Get the effective Hamiltonian
ham = vqls.effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)

# Display the effective Hamiltonian
nterms = len(ham)
print("Number of terms:", nterms)
print("H = ")
for ii in range(nterms):
    ham[ii][0] = np.real(ham[ii][0])
    print(f"{round(ham[ii][0], 3)} \t* \t{ham[ii][1]}", (lambda ii: "+" if ii < nterms - 1 else "")(ii))

In [ ]:
"""Group Hamiltonian into simultaneously measurable sets of operators."""
ham = vqls.group_greedy(ham)
print("# groups =", len(ham))

# Consistency check
for group in ham:
    _, paulis = vqls.split_ham_to_coeffs_and_paulis(group)
    assert vqls.is_sim_meas_group(paulis)

In [ ]:
"""Get a quantum computer to run on."""
qcomputer = f"Aspen-7-{n}Q-B"  # Three qubit lattice for testing optimizer
lattice = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

In [ ]:
"""Visualize the qubit connectivity."""
graph = lattice.qubit_topology()
nx.draw(graph)

In [ ]:
"""Get an ansatz and visualize it."""
circ, creg = vqls.yansatzCZ(lattice)
print(circ)

In [ ]:
"""Cost function."""
def cost(angles, verbose: bool = False):
    val = vqls.energy_sim(angles, ham, circ, creg, lattice, shots=10_000)
    if verbose:
        print("Current angles:", angles)
        print("Current energy:", val)
    return val

In [ ]:
"""Do the optimization."""
start = time.time()
res = minimize(cost, x0=[pi / 2] * 3 * n, method="Powell", args=(True,))
print("Runtime:", (time.time() - start) // 60, "minutes.")

In [ ]:
print("Minimum cost found:", cost(res))
print("Best found angles:", res)

In [ ]:
qxvec = vqls.qsolution(circ, res)
print(qxvec)

In [ ]:
"""Compare to classical solution."""
# Get the classical solution
Amat = vqls.matrix(Acoeffs, Aterms)
bvec = vqls.vector(Bcoeffs, Bterms)
xvec = np.linalg.solve(Amat, bvec)
xvec /= np.linalg.norm(xvec)

# ===========================
# Compare classical x vectors
# ===========================

print("Comparing x vectors:")
print("====================")

# Compute the fidelity
fidelity = abs(np.dot(qxvec.conj(), xvec))**2
print(r"| < xquantum | xclassical > |^2 =", round(fidelity, 2))

# Compute the L2 distance
l2dist = np.linalg.norm(qxvec - xvec, ord=2)**2
print(r"|| xquantum - xclassical ||^2 =", round(l2dist, 2))

# ==========================
# Compare solution b vectors
# ==========================

print("\nComparing b vectors:")
print("====================")

# Compute |qbvec> = A |theta optimal>
qbvec = Amat @ qxvec
qbvec /= np.linalg.norm(qbvec)
bfidelity = abs(np.dot(bvec.conj(), qbvec))**2
print(r"| < bquantum | bclassical > |^2 =", round(bfidelity, 2))

bl2dist = np.linalg.norm(qbvec - bvec, ord=2)**2
print(r"|| bquantum - bclassical ||^2 =", round(bl2dist, 2))

# <p style="text-align: center;"> Linear System 2 </p>

The second linear system we consider is formed from the Ising model.

\begin{equation}
    A_2 := \frac{1}{\zeta} \left(\eta I + \sum_{j=1}^{n} X_j + J \sum_{j=1}^{n-1}Z_jZ_{j+1} \right)
\end{equation}

In [7]:
def get_Acoeffs_Aterms_LS2(n: int, zeta: float, eta: float, J: float):
    """Returns linear system 2 expressed as weighted Pauli sum."""
    if n < 0:
        raise ValueError("Number of qubits must be >= 1.")
    # Add the identity
    Acoeffs = [eta / zeta]
    Aterms = ["I" * n]
    
    # Add the X terms
    Acoeffs += [1 / zeta] * n
    xbase = "X" + "I" * (n - 1)
    for ii in range(n, 0, -1):
        Aterms.append(xbase[ii:] + xbase[:ii])
        
    # Add the ZZ terms
    Acoeffs += [J / zeta] * (n - 1)
    zzbase = "ZZ" + "I" * (n - 2)
    for ii in range(n, 1, -1):
        Aterms.append(zzbase[ii:] + zzbase[:ii])
    
    return Acoeffs, Aterms

In [8]:
"""Unit tests for getting LS2."""
Acoeffs, Aterms = get_Acoeffs_Aterms_LS2(4, 1, 1, 0.1)
assert Acoeffs == [1.0, 1.0, 1.0, 1.0, 1.0, 0.1, 0.1, 0.1]
assert Aterms == ['IIII', 'XIII', 'IXII', 'IIXI', 'IIIX', 'ZZII', 'IZZI', 'IIZZ']

In [9]:
"""Terms in effective Hamiltonian for second linear system."""
# Number of qubits
n = 3

# Constants
zeta = 1.0
J = 0.1
eta = 1.0

# |b> = H|0>
# Bcoeffs = [1 / 2**(n / 2)] * 2**n
# paulis = [["X", "Z"]] * n
# prods = list(product(*paulis))
# Bterms = ["".join(p) for p in prods]

# |b> = |10...0>
Bcoeffs = [1.]
Bterms = ["I" * n]

# Linear system in weighted Paulis
Acoeffs, Aterms = get_Acoeffs_Aterms_LS2(n, zeta, eta, J)

# Normalize A and B coefficients
# TODO: Do this calculation in terms of the coefficients, not the explicit matrix
Amat = vqls.matrix(Acoeffs, Aterms)
Anorm = np.linalg.norm(Amat, ord=2)
Acoeffs = np.array(Acoeffs)
Acoeffs /= Anorm

Bmat = vqls.matrix(Bcoeffs, Bterms)
Bnorm = np.linalg.norm(Bmat, ord=2)
Bcoeffs = np.array(Bcoeffs)
Bcoeffs /= Bnorm

ham = vqls.effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)

# Display the effective Hamiltonian
nterms = len(ham)
num = 0
min_weight = 0.01
print("Number of terms:", nterms)
for ii in range(nterms):
    ham[ii][0] = round(np.real(ham[ii][0]), 3)
    if abs(ham[ii][0]) > min_weight:
        num += 1
#     print(f"{np.real(round(ham[ii][0], 3))} \t* \t{ham[ii][1]}", (lambda ii: "+" if ii < nterms - 1 else "")(ii))

print(f"Number of terms with |coeff| > {min_weight} = {num}")

Number of terms: 32
Number of terms with |coeff| > 0.01 = 27


In [10]:
"""View matrix of linear system we are solving."""
Amat = vqls.matrix(Acoeffs, Aterms)
print("A =\n", np.real(Amat))

A =
 [[0.3  0.25 0.25 0.   0.25 0.   0.   0.  ]
 [0.25 0.25 0.   0.25 0.   0.25 0.   0.  ]
 [0.25 0.   0.2  0.25 0.   0.   0.25 0.  ]
 [0.   0.25 0.25 0.25 0.   0.   0.   0.25]
 [0.25 0.   0.   0.   0.25 0.25 0.25 0.  ]
 [0.   0.25 0.   0.   0.25 0.2  0.   0.25]
 [0.   0.   0.25 0.   0.25 0.   0.25 0.25]
 [0.   0.   0.   0.25 0.   0.25 0.25 0.3 ]]


In [27]:
"""Drop terms with weight below a given threshold."""
def drop_below(threshold: float):
    ham_thresh = []
    for ii in range(len(ham)):
        if abs(ham[ii][0]) > threshold:
            ham_thresh.append(ham[ii])
    return ham_thresh

In [28]:
"""Drop Paulis in Hamiltonian with weight below a given threshold."""
threshold = 0.01
hamt = drop_below(threshold)
print("# terms =", len(hamt))

# terms = 37


In [29]:
"""Group the Hamiltonian into simultaneously measurable sets."""
ham = vqls.group_greedy(ham, randomized=True)
print("# groups =", len(ham))

# groups = 124


In [30]:
hamt = vqls.group_greedy(hamt, randomized=True)
print("# groups =", len(hamt))

# groups = 1


In [31]:
"""Get a computer to run on."""
qcomputer = f"Aspen-7-{n}Q-B"
lattice = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

In [32]:
"""Get an ansatz."""
circ, creg = vqls.yansatz(lattice)
print(circ)

DECLARE ro BIT[8]
DECLARE theta REAL[8]
RY(theta[0]) 21
RY(theta[1]) 30
RY(theta[2]) 31
RY(theta[3]) 32
RY(theta[4]) 33
RY(theta[5]) 35
RY(theta[6]) 36
RY(theta[7]) 37



In [33]:
"""Cost function for linear system 2."""
def cost(angles, verbose: bool = False):
    val = vqls.energy_sim(angles, hamt, circ, creg, lattice, shots=10_000)
    if verbose:
        print("Current angles:", angles)
        print("Current energy:", val)
    return val

In [ ]:
# Good initial paramaters for the 8 qubit Ising linear system
warm_start = [-1.046,  0.065, -0.842, -1.556, -0.442,  2.197,  0.738, -0.967]

In [35]:
"""Do the optimization."""
start = time.time()
res = minimize(cost, x0=np.random.randn(n), method="Powell", args=(True,))
print("Runtime:", (time.time() - start) // 60, "minutes.")

Current angles: [ 1.623  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.142728
Current angles: [ 1.623  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.14004960000000002
Current angles: [ 2.623  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.1171536
Current angles: [ 4.241  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.05517599999999999
Current angles: [ 6.859  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.11973599999999997
Current angles: [ 4.241  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.054235200000000004
Current angles: [ 5.241  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.05708640000000003
Current angles: [ 3.623  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.07537440000000002
Current angles: [ 4.679  1.211  0.404 -0.356 -0.856  0.939  0.841 -1.163]
Current energy: 0.04997279999999997
Current angles: [ 4.648  1.211

Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.551  0.841 -1.163]
Current energy: 0.019080000000000007
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557  0.841 -1.163]
Current energy: 0.01898399999999999
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557  1.841 -1.163]
Current energy: 0.01923360000000001
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557 -0.777 -1.163]
Current energy: 0.01713120000000001
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557 -3.395 -1.163]
Current energy: 0.018264000000000002
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557 -0.777 -1.163]
Current energy: 0.0169488
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557 -1.777 -1.163]
Current energy: 0.016574399999999986
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557 -2.395 -1.163]
Current energy: 0.017015999999999986
Current angles: [ 4.679  1.513 -1.681  1.825 -1.474  1.557 -1.555 -1.163]
Current energy: 0.016632000000000008
Current angles:

Current angles: [ 4.638  1.895 -1.573  1.825 -1.474  1.557 -1.741  1.481]
Current energy: 0.016027199999999995
Current angles: [ 4.638  1.895 -1.574  1.825 -1.474  1.557 -1.741  1.481]
Current energy: 0.016132800000000003
Current angles: [ 4.638  1.895 -1.573  1.825 -1.474  1.557 -1.741  1.481]
Current energy: 0.016152000000000007
Current angles: [ 4.638  1.895 -1.573  2.825 -1.474  1.557 -1.741  1.481]
Current energy: 0.016372799999999983
Current angles: [ 4.638  1.895 -1.573  0.207 -1.474  1.557 -1.741  1.481]
Current energy: 0.01682399999999998
Current angles: [ 4.638  1.895 -1.573  1.825 -1.474  1.557 -1.741  1.481]
Current energy: 0.016075199999999987
Current angles: [ 4.638  1.895 -1.573  1.207 -1.474  1.557 -1.741  1.481]
Current energy: 0.01601759999999998
Current angles: [ 4.638  1.895 -1.573  0.825 -1.474  1.557 -1.741  1.481]
Current energy: 0.01622880000000001
Current angles: [ 4.638  1.895 -1.573  1.443 -1.474  1.557 -1.741  1.481]
Current energy: 0.0159888
Current angles:

Current angles: [ 4.638  1.895 -1.573  1.49  -1.564  1.627 -1.574  1.558]
Current energy: 0.015163200000000002
Current angles: [ 4.638  1.895 -1.573  1.49  -1.564  1.627 -1.574  1.561]
Current energy: 0.015105599999999976
Current angles: [ 4.638  1.895 -1.573  1.49  -1.564  1.627 -1.574  1.564]
Current energy: 0.015095999999999998
Current angles: [ 4.638  1.895 -1.573  1.49  -1.564  1.627 -1.574  1.562]
Current energy: 0.015153599999999993
Current angles: [ 4.597  2.277 -1.466  1.155 -1.655  1.696 -1.407  1.644]
Current energy: 0.016766400000000008
Current angles: [ 4.638  1.895 -1.573  1.49  -1.564  1.627 -1.574  1.562]
Current energy: 0.015230400000000005
Current angles: [ 5.638  1.895 -1.573  1.49  -1.564  1.627 -1.574  1.562]
Current energy: 0.0339312
Current angles: [ 3.02   1.895 -1.573  1.49  -1.564  1.627 -1.574  1.562]
Current energy: 0.06690719999999997
Current angles: [ 4.638  1.895 -1.573  1.49  -1.564  1.627 -1.574  1.562]
Current energy: 0.015124800000000011
Current angle

Current angles: [ 4.706  1.581 -1.584  2.396 -1.579  1.627 -1.574  1.562]
Current energy: 0.015009600000000012
Current angles: [ 4.706  1.581 -1.584  2.396 -1.582  1.627 -1.574  1.562]
Current energy: 0.015028800000000009
Current angles: [ 4.706  1.581 -1.584  2.396 -1.583  1.627 -1.574  1.562]
Current energy: 0.015028800000000002
Current angles: [ 4.706  1.581 -1.584  2.396 -1.583  1.627 -1.574  1.562]
Current energy: 0.015028799999999995
Current angles: [ 4.706  1.581 -1.584  2.396 -1.583  2.627 -1.574  1.562]
Current energy: 0.018868800000000012
Current angles: [ 4.706  1.581 -1.584  2.396 -1.583  0.009 -1.574  1.562]
Current energy: 0.023006400000000003
Current angles: [ 4.706  1.581 -1.584  2.396 -1.583  1.627 -1.574  1.562]
Current energy: 0.015038400000000007
Current angles: [ 4.706  1.581 -1.584  2.396 -1.583  1.009 -1.574  1.562]
Current energy: 0.016219200000000003
Current angles: [ 4.706  1.581 -1.584  2.396 -1.583  2.009 -1.574  1.562]
Current energy: 0.01569119999999999
Cu

Current angles: [ 4.708  1.581 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.015
Current angles: [ 4.708  1.581 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.015
Current angles: [ 4.708  1.581 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.015019200000000024
Current angles: [ 4.708  2.581 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.01854240000000002
Current angles: [ 4.708 -0.037 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.023380800000000014
Current angles: [ 4.708  1.581 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.015009599999999998
Current angles: [ 4.708  0.963 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.01631519999999999
Current angles: [ 4.708  1.963 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.01567200000000002
Current angles: [ 4.708  1.546 -1.584  2.396 -1.583  1.553 -1.573  1.582]
Current energy: 0.0150192
Current angles: [ 4.708  1.592 -1.584  2.396 

Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015019199999999983
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015009599999999998
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015009599999999998
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015019199999999996
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015019199999999996
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015028799999999988
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.014999999999999986
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015009599999999991
Current angles: [ 4.708  1.602 -1.575  3.205 -1.583  1.553 -1.573  1.582]
Current energy: 0.015028800000000016
C

KeyboardInterrupt: 

In [ ]:
# """Optimize by gradient descent."""
# optimizer = AQGD(maxiter=25, tol=1e-3, eta=5.0, momentum=0.25, disp=True)
# start = time.time()
# res = optimizer.optimize(np.random.randn(n), costLS2)
# print("Runtime:", (time.time() - start) // 60, "minutes.")

In [39]:
"""Optimization: Testing fmin functions from scipy.optimize."""
start = time.time()
res = fmin_powell(cost, x0=[pi / 2] * n, maxfun=50, args=(True,))
print("Runtime:", (time.time() - start) // 60, "minutes.")

Current angles: [1.571 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.9750000000000005
Current angles: [1.571 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.9750000000000005
Current angles: [2.571 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.8849136000000001
Current angles: [4.189 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.6162288000000001
Current angles: [6.807 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.8809968000000002
Current angles: [4.189 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.6153072000000002
Current angles: [5.189 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.6134256000000002
Current angles: [5.807 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.6941808000000006
Current angles: [4.7   1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.5910000000000002
Current angles: [4.669 1.571 1.571 1.571 1.571 1.571 1.571 1.571]
Current energy: 0.5910768

Current angles: [4.7   4.704 4.722 4.725 1.262 1.584 1.605 1.571]
Current energy: 0.015028800000000023
Current angles: [4.7   4.704 4.722 4.725 1.262 1.584 1.605 0.953]
Current energy: 0.015249600000000012
Current angles: [4.7   4.704 4.722 4.725 1.262 1.584 1.605 1.953]
Current energy: 0.01511520000000002
Current angles: [4.7   4.704 4.722 4.725 1.262 1.584 1.605 1.568]
Current energy: 0.015028800000000026
Current angles: [4.7   4.704 4.722 4.725 1.262 1.584 1.605 1.569]
Current energy: 0.015009599999999991
Current angles: [4.7   4.704 4.722 4.725 1.262 1.584 1.605 1.569]
Current energy: 0.015009599999999994
Current angles: [4.7   4.704 4.722 4.725 1.262 1.584 1.605 1.569]
Current energy: 0.01502880000000003
Runtime: 5.0 minutes.


In [40]:
print("Optimal angles:", res % (np.pi))

Optimal angles: [1.559 1.562 1.58  1.584 1.262 1.584 1.605 1.569]


In [41]:
"""Evaluate the cost using the complete (untruncated) Hamiltonian."""
copt = vqls.energy_sim(res, ham, circ, creg, lattice, shots=10_000)
print("Full optimal cost:", copt)

Full optimal cost: 0.014419999999999954


In [ ]:
"""Get the quantum solution (x vector)."""
qxvec = vqls.qsolution(circ, res.x)
print(*np.real(qxvec), sep="\n")

In [ ]:
"""Compare to classical solution."""
# Get the classical solution
Amat = vqls.matrix(Acoeffs, Aterms)
bvec = vqls.vector(Bcoeffs, Bterms)
xvec = np.linalg.solve(Amat, bvec)
xvec /= np.linalg.norm(xvec)

# ===========================
# Compare classical x vectors
# ===========================

print("Comparing x vectors:")
print("====================")

# Compute the fidelity
fidelity = abs(np.dot(qxvec.conj(), xvec))**2
print(r"| < xquantum | xclassical > |^2 =", round(fidelity, 2))

# Compute the L2 distance
l2dist = np.linalg.norm(qxvec - xvec, ord=2)**2
print(r"|| xquantum - xclassical ||^2 =", round(l2dist, 2))

# ==========================
# Compare solution b vectors
# ==========================

print("\nComparing b vectors:")
print("====================")

# Compute |qbvec> = A |theta optimal>
qbvec = Amat @ qxvec
qbvec /= np.linalg.norm(qbvec)
bfidelity = abs(np.dot(bvec.conj(), qbvec))**2
print(r"| < bquantum | bclassical > |^2 =", round(bfidelity, 2))

bl2dist = np.linalg.norm(qbvec - bvec, ord=2)**2
print(r"|| bquantum - bclassical ||^2 =", round(bl2dist, 2))

## Running on a Quantum Computer

The experiments in this notebook are made into a script and run on Rigetti's Quantum Cloud Services. In particular, we used the Aspen-7 chip, the fidelity/noise characterization of which can be found online. In the following cells, we plot our results.

## Error Mitigation

The QPU data above has a significant vertical shift from the simulator data, but generally the same "shape." This shift is present because of decoherence, gate application errors, measurement errors, and other noise in the system. It can be accounted for by running a set of "benchmark circuits" on the QPU prior to running the actual algorithm. These benchmark circuits are simple circuits, such as NOT and MEASURE, for which one knows the actual output. The vertical shift in this benchmark circuit can then be tested for and subtracted from the final computed energies to get more accurate results. Other methods are also possible, and this is an area of active research.

Here, we employ a similar idea, but instead of running a benchmark circuit, we just subtract off the difference of the initial energies on the QPU and on the simulator. The cell below implements this and plots the result again.

In [ ]:
"""Plot the error mitigated QPU energy vs iteration data obtained by running on Rigetti QCS."""
# Constant shift amount. In practice, this would be obtained by running a "benchmark circuit."
# Here, we just set the value based on the above curves
shift = 0.3

# Read in the files
qpu_energy1 = np.loadtxt("qpu-energy-iteration1.txt")
qpu_energy2 = np.loadtxt("qpu-energy-iteration2.txt")

# Subtract off the shift
qpu_energy1 -= shift
qpu_energy2 -= shift

# Do the plotting
plt.figure(figsize=(10, 5))
plt.plot(energies, "--o", linewidth=3, label="Simulator")
plt.xlabel("Iteration", fontsize=14, fontweight="bold")
plt.ylabel("Energy", fontsize=14, fontweight="bold")

plt.plot(qpu_energy1, "--o", linewidth=3, label="Error Mitigated QPU Run 1")
plt.plot(qpu_energy2, "--o", linewidth=3, label="Error Mitigated QPU Run 2")

# Put a line at the actual ground state energy (see below)
GSENERGY = 0.53232723
plt.plot(GSENERGY * np.ones_like(energies), linewidth=3, label="Analytic Energy")

plt.grid()
plt.legend();

## Conclusions

## Acknowledgements

Thank coauthors and Rigetti.

## References

[1] Carlos Bravo-Prieto, Ryan LaRose, M. Cerezo, Yigit Subasi, Lukasz Cincio, Patrick J. Coles, [Variational quantum linear solver: A hybrid algorithm for linear systems](https://arxiv.org/abs/1909.05820), 2019.

[2] Aram W. Harrow, Avinatan Hassidim, Seth Lloyd, [Quantum algorithm for solving linear systems of equations](https://arxiv.org/abs/0811.3171), 2008.

[3] Leonard Wossnig, Zhikuan Zhao, Anupam Prakash, [A quantum linear system algorithm for dense matrices](https://arxiv.org/abs/1704.06174), 2017.

[4] Yigit Subasi, Rolando Somma, David Orsucci, [Quantum algorithms for systems of linear equations inspired by adiabatic quantum computing](https://arxiv.org/abs/1805.10549), 2018.

[5] Alberto Peruzzo, Jarrod McClean, Peter Shadbolt, Man-Hong Yung, Xiao-Qi Zhou, Peter J. Love, Alán Aspuru-Guzik, Jeremy L. O'Brien, [A variational eigenvalue solver on a quantum processor](https://arxiv.org/abs/1304.3061), 2014.

[6] Jarrod R. McClean, Jonathan Romero, Ryan Babbush, Alán Aspuru-Guzik, [The theory of variational hybrid quantum-classical algorithms](https://arxiv.org/abs/1509.04279), 2015. 

[7] Ryan LaRose et al., [Implementing variational quantum algorithms](https://github.com/rmlarose/qcbq/blob/master/tutorials/04qaoa/qcbq_tutorial4b_implement_qaoa-INSTRUCTOR.ipynb), MSU-IBM Quantum computing bootcamp with Qiskit, 2019. 